# Parser/ Exploration of new file formats from Cindy Oakley
February 23, 2015

### Outline
1. Preliminaries
    * load libraries and helper functions
    * define useful dictionaries relating states to state-codes and names
    * define listings of coverages by state
2. Read in the raw data files
    * process the meta-data as column headers
    * clean up the column header names
    * create a timestamp for the dates based upon month & year columns
    * keep the Losses file

3. Process the losses file
    * drop Company wide and Zone totals
    * drop sub-sections of big states (NY, CA, FL, TX) in lieu of totaled values
    * keep only total voluntary losses: LINE=='TOTVOL'
    * zero out the PIP or MPC values for states that lack these coverages.
    * Calculate derived fields:
        * Injury = BI+UBI+WBI
        * PIP/MPC = PIP+MPC
        * Property = PD+COLL+COMP
        * Severity = Paid_Amt / Paid_Count
    * Drop unused columns (for now)
        * CWP
        * SUIT_CNT
        * ALAE
        * OIE_CNT
        * MONTH/YEAR?
    * save as csv


## 1. Preliminaries

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from collections import defaultdict

In [ ]:
def printall(X, max_rows=10):
    from IPython.display import display, HTML
    display(HTML(X.to_html(max_rows=max_rows)))

In [ ]:
# create a dictionary for the STATECODES
# dictionary mapping state_codes to state-abbreviations
stateDict ={ '01':'AL', '02':'AK', '03':'AZ', '04':'AR', '05':'CA', '06':'CO', '07':'CT', '08':'DE', '09':'DC', 
             '11':'GA', '12':'ID', '13':'IL', '14':'IN', '15':'IA', '16':'KS', '17':'KY', '18':'LA', '19':'ME',
             '20':'MD', '22':'MI', '23':'MN', '24':'MS', '25':'MO', '26':'MT', '27':'NE', '28':'NV', '29':'NH', 
             '30':'NJ', '31':'NM', '32':'NY', '33':'NC', '34':'ND', '35':'OH', '36':'OK', '37':'OR', '38':'PA', 
             '40':'SC', '41':'SD', '42':'TN', '43':'TX', '44':'UT', '45':'VT', '46':'VA', '47':'WA', '48':'WV', 
             '49':'WI', '50':'WY', '51':'HI', '52':'NY', '53':'TX', '55':'CA', '59':'FL', '75':'CA', 
             '21':'MA', '39':'RI',
             '94':'TX','96':'CA','97':'NY'}
print len(stateDict)

In [ ]:
stateNamesDict = {'01': 'ALABAMA', '02': 'ALASKA', '03':'ARIZONA','04':'ARKANSAS','06':'COLORADO',
                  '07':'CONNECTICUT','08' : 'DELAWARE','09':'DIST. OF COL.','11':'GEORGIA','12':'IDAHO',    
#when state = '05' then 'CALIF N COAST'       
                  '13':'ILLINOIS','14':'INDIANA','15':'IOWA','16':'KANSAS','17':'KENTUCKY',
                  '18': 'LOUISIANA','19' : 'MAINE','20' : 'MARYLAND','21' : 'MASSACHUSETTS','22' : 'MICHIGAN',       
                  '23' : 'MINNESOTA','24' : 'MISSISSIPPI','25':'MISSOURI','26' :'MONTANA',            
                  '27':'NEBRASKA','28':'NEVADA','29':'NEW HAMPSHIRE','30':'NEW JERSEY','31':'NEW MEXICO',
                  #'32':'NEW YORK METRO'      
                  '33':'NORTH CAROLINA','34':'NORTH DAKOTA','35':'OHIO','36':'OKLAHOMA','37':'OREGON',
                  '38':'PENNSYLVANIA','39':'RHODE ISLAND','40':'SOUTH CAROLINA','41':'SOUTH DAKOTA','42':'TENNESSEE',
                  #'43':'TEXAS NO.'           
                  '44':'UTAH','45':'VERMONT','46':'VIRGINIA','47':'WASHINGTON','48':'WEST VIRGINIA',
                  '49':'WISCONSIN','50':'WYOMING','51':'HAWAII',              #'52':'NEW YORK HERIT'      
                  #'53':'TEXAS SO.'           #'55':'CALIF. GR.'          
                  '59':'FLORIDA',#'95':'FLORIDA',
                  '94':'TEXAS','96':'CALIFORNIA','97':'NEW YORK'}
                    #'75':'CALIF S COAST' 

In [ ]:
# list of pip/mpc/both 
pipCoverageMap = defaultdict(list)
both_state_list = ['FLORIDA','MASSACHUSETTS','TEXAS','VIRGINIA']
mpc_state_list = ['ALABAMA',
 'ALASKA',
 'ARIZONA',
 'ARKANSAS',
 'CALIFORNIA',
 'COLORADO',
 'CONNECTICUT',
 'GEORGIA',
 'IDAHO',
 'ILLINOIS',
 'INDIANA',
 'IOWA',
 'LOUISIANA',
 'MAINE',
 'MISSISSIPPI',
 'MISSOURI',
 'MONTANA',
 'NEBRASKA',
 'NEVADA',
 'NEW HAMPSHIRE',
 'NEW MEXICO',
 'NORTH CAROLINA',
 'OHIO',
 'OKLAHOMA',
 'RHODE ISLAND',
 'SOUTH DAKOTA',
 'TENNESSEE',
 'VERMONT',
 'WEST VIRGINIA',
 'WISCONSIN',
 'WYOMING']
pip_state_list = [
 'DELAWARE',
 'DIST. OF COL.',
 'HAWAII',
 'KANSAS',
 'KENTUCKY',
 'MARYLAND',
 'MICHIGAN',
 'MINNESOTA',
 'NEW JERSEY',
 'NEW YORK',
 'NORTH DAKOTA',
 'OREGON',
 'PENNSYLVANIA',
 'SOUTH CAROLINA',
 'UTAH',
 'WASHINGTON']

## 2. Read in the raw data files

In [ ]:
%cd '../../../projects/CENT/dataSets/2015-01/'
%ls

In [ ]:
# glob the files
from glob import glob
flist = glob('*.txt')
len(flist)

In [ ]:
## process the meta-data as column headers
meta_line = []
import re

with open(flist[0]) as infile:
    for line in infile:
        if len(line)>1:
            line=line.strip('\n')#meta_line.append(line.strip('\n'))
            #print line.split()
            #print line.strip('\n').split(None,2)
            tline = re.sub("     ", "\t", line)
            elements = tline.split('\t')
            elements = [x.strip() for x in elements]
            meta_line.append(elements)


In [ ]:
# now process these meta_line values
from collections import defaultdict
meta_names = defaultdict(list)
for row in meta_line[1:]:
    if len(row)==1:
        #print row
        infile_names = row[0].split(' File')[:-1]
        infile_names = [x.strip() for x in infile_names]
        print infile_names
    elif len(row)==3:
        for j in xrange(0,3):
            meta_names[infile_names[j]].append(row[j])
    elif len(row)>3:
        if len(row[0]):
            meta_names[infile_names[0]].append(row[0])
        meta_names[infile_names[2]].append(row[-1])
        

In [ ]:
for k,v in meta_names.iteritems():
    print k,v, len(v)

In [ ]:
## so these are the headers for each of the next set of files
input_df = []
for f in flist[1:]:
    # convert to dict_key
    my_key = f[11:14]
    if my_key == 'Los':
        my_key = 'Loss'
    print "opening file {0} with {1} columns".format(f, len(meta_names[my_key]))
    ## now open that file using pandas
    input_df.append(pd.read_csv(f,header=None,sep='~',names=meta_names[my_key]))
    
print [df.shape for df in input_df]

### now clean up these dataframes
 #### begin by condensing the name values in the LINE & Coverage columns


In [ ]:
## create a dictionary for each 
uniq_line_name_dict = {}
uniq_coverage_name_dict = {}
for adf in input_df:
    uline_names = list(adf.LINE.unique())
    if 'COVERAGE' in list(adf.columns):
        ucoverage_names = list(adf.COVERAGE.unique())
        for y in ucoverage_names:
            uniq_coverage_name_dict[y] = y.strip()
    
    for x in uline_names:
        uniq_line_name_dict[x]= x.strip()
    
        
print uniq_line_name_dict
print uniq_coverage_name_dict

#### Now use these dictionaries to replace the values in each dataframe


In [ ]:
for i in xrange(0,len(input_df)):
    input_df[i].replace(to_replace=uniq_line_name_dict,inplace=True)
    if 'COVERAGE' in list(input_df[i].columns):
        input_df[i].replace(to_replace=uniq_coverage_name_dict,inplace=True)
    

In [ ]:
print input_df[1].COVERAGE.unique(), input_df[1].LINE.unique()

### Next combine YEAR+MONTH to datetime (beginning of month)


In [ ]:
## create new column 'date' in these cases
for i in xrange(0,len(input_df)):
    input_df[i]['date'] =input_df[i][['MONTH','YEAR']].apply(lambda x: pd.to_datetime("-".join(map(str,x)),format='%m-%Y'),axis=1)

In [ ]:
printall(input_df[1])

## 3. Process the losses file

### Separate out the aggregated values for Company wide (CW), zones (ZN), and special states 

* California --> total is 96; drop 05,55,75
* Texas --> total is 94; drop 43,53
* New York --> toal is 97; drop 32, 52
   

In [ ]:
# first separate out the aggregated values for Company wide (CW) and zones (ZN)
input_df[0].STATE.value_counts()

In [ ]:
cif_df = input_df[0][~input_df[0].STATE.isin(['CW','ZN'])].copy()
loss_df = input_df[1][~input_df[1].STATE.isin(['CW','ZN'])].copy()
pif_df = input_df[2][~input_df[2].STATE.isin(['CW','ZN'])].copy()

In [ ]:
## drop TX: 43,53; CA: 05,55,75; NY: 32,52
substate_codes_to_drop = ['43','53','05','55','75','32','52']
cif_df = cif_df[~cif_df.STATE.isin(substate_codes_to_drop)].copy()
loss_df = loss_df[~loss_df.STATE.isin(substate_codes_to_drop)].copy()
pif_df = pif_df[~pif_df.STATE.isin(substate_codes_to_drop)].copy()
print len(cif_df),len(pif_df),len(loss_df)

In [ ]:
### drop MONTH & YEAR columns
for i in xrange(0,len(input_df)):
    input_df[i].drop(['YEAR','MONTH'],axis=1,inplace=True)

### Aside dealing with CIF file

In [ ]:
cif_df[cif_df.columns[5:-1]].sum(axis=1) - cif_df.ALL

In [ ]:
cif_df.groupby('STATE')[['MPC','PIP']].sum()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(cif_df[cif_df.columns[5:-1]].sum(axis=1), cif_df.ALL)
x = np.linspace(0,20000000)
y = x
plt.plot(x,y,color='red')
plt.xlabel('sum CIF_columns')
plt.ylabel('ALL for CIF')

In [ ]:
loss_df.LINE.unique()

In [ ]:
(loss_df[((loss_df.STATE=='05') |(loss_df.STATE=='55') | (loss_df.STATE=='75')) & (loss_df.LINE =='TOTVOL') & (loss_df.YEAR ==2007) & (loss_df.MONTH == 1)]).sort('COVERAGE')

In [ ]:
# okay so at least for TX '43'+'53' yields '94'

In [ ]:
loss_df[(loss_df.STATE=='96') & (loss_df.LINE =='TOTVOL ') & (loss_df.YEAR ==2007) & (loss_df.MONTH == 1)].sort('COVERAGE')

In [ ]:
(loss_df[((loss_df.STATE=='05') |(loss_df.STATE=='55') | (loss_df.STATE=='75')) & (loss_df.LINE =='TOTVOL ') & (loss_df.YEAR ==2007) & (loss_df.MONTH == 1)]).groupby('COVERAGE').sum()

In [ ]:
loss_df.COVERAGE.unique()

In [ ]:
loss_df[(loss_df.STATE=='01') & (loss_df.YEAR==2009)&(loss_df.MONTH<6) & (loss_df.COVERAGE=='BI ')]

In [ ]:
loss_df.COVERAGE.unique()

### keep only the TOTAL VOLUNTARY AUTO POLICIES


In [ ]:
volloss= loss_df[loss_df.LINE=='TOTVOL'].copy()
volloss.drop(['ZONE','LINE'],inplace=True,axis=1)#.sort('COVERAGE',inplace=True)
# & (loss_df.COVERAGE==')][alabama_loss_temp.columns[3:]]
volloss

In [ ]:
pd.date_range('01-01-2007','02-01-2015',freq='M')

In [ ]:
volloss.columns

### replace state codes with abbreviations/names

In [ ]:
volloss.replace(to_replace={'STATE':stateNamesDict},inplace=True)

In [ ]:
volloss.sort(['STATE','COVERAGE','date'],inplace=True)

In [ ]:
saveRaw = False
if saveRaw = True:
    volloss.to_csv('totalvol_loss_raw.csv',index=False)


### "Correct" for cases where certain coverages are not offered
Logic is to zero out the values if PIP or MPC is not available in that state

In [ ]:

def zero_missing_coverage(df,coverage,states_list,
                          cols_to_zero= ['REPORTED_CNT','PAID_CNT','PENDING_CNT','CWP','OIE_CNT','SUIT_CNT','PD_AMT','ALAE']):
    zero_index = df[(df.COVERAGE==coverage)&df.STATE.isin(states_list)].index
    df.loc[zero_index,cols_to_zero]=0
    print len(zero_index)
    return df
#idx=3
#mtrc='REPORTED_CNT'
#sub_df = volloss[(volloss.COVERAGE==cvr) & (volloss.STATE == mpc_state_list[idx])]


In [ ]:
vv = zero_missing_coverage(volloss, 'MPC',pip_state_list)
vv = zero_missing_coverage(vv,'PIP',mpc_state_list)


### Drop those called COVERAGE == 'ALL'


In [ ]:
vv = vv[vv.COVERAGE!='ALL'].copy()
vv.shape

### Drop columns not previously used:
* ALAE
* SUIT_CNT
* OIE_CNT
* CWP

In [ ]:
vv.columns

In [ ]:
cols_to_drop =  ['CWP','OIE_CNT','SUIT_CNT','ALAE','YEAR','MONTH']
col_to_rename ={'REPORTED_CNT':'Reported Count','PAID_CNT':'Paid Count','PENDING_CNT':'Pending Count','PD_AMT':'Indemnity'}#,'date':'YEAR'}

In [ ]:
vv.drop(cols_to_drop,axis=1, inplace=True)
vv.rename(columns=col_to_rename,inplace=True)
print vv.columns
print vv.shape

## Populate with Derived/Calculated Fields:

2. Injury = BI+UBI+WBI
3. PIP/MPC = PIP+MPC
4. Property = PD+COMP+COLL
1. Severity = Indemnity (PD_AMT) / PAID_CNT

# define the combined coverages
combo_coverages = ['Injury','Property','PIP/MPC']

In [ ]:
def return_summed_coverages(df,coverage_list,coverage_name):
    new_df = df[df.COVERAGE.isin(coverage_list)].groupby(('STATE','date'),as_index=False).sum()
    new_df['COVERAGE']=coverage_name
    return new_df

In [ ]:
injury_df = return_summed_coverages(vv,['BI','UBI','WBI'],'Injury')
property_df = return_summed_coverages(vv,['PD','COLL','COMP'],'Property')
pipmpc_df = return_summed_coverages(vv,['PIP','MPC'],'PIP/MPC')
vv = pd.concat([vv,injury_df,property_df,pipmpc_df])
#vv.append(adf)
print vv.shape

In [ ]:
vv.COVERAGE.unique()

#### Calculate Severity (and replace inf/-inf with zero)

In [ ]:
vv['Severity']=vv['Indemnity']/vv['Paid Count']#volloss['PD_AMT']/volloss['PAID_CNT']

In [ ]:
#sum(vv[vv.Severity==np.inf])
vv.replace([np.inf, -np.inf], 0.0,inplace=True)
vv.Severity.value_counts()

In [ ]:
# reorder the columns to match previous dataset order
vcols = vv.columns
print vcols
mod_order = [vcols[-3],vcols[-2],vcols[0],vcols[4],vcols[2],vcols[3],vcols[1],vcols[-1]]
print mod_order
vv=vv[mod_order]
vv.rename(columns={'date':'YEAR'},inplace=True)
vv.head()

In [ ]:
#sort this by state/coverage/year
vv.sort(['STATE','COVERAGE','YEAR'],inplace=True)
# relabel the indices
vv.index=xrange(0,len(vv))
vv.head()

In [ ]:
### Save the file
saveDerived = True
if saveDerived:
    vv.to_csv('base_CENT_01_2015.csv',index=False)



In [ ]:
vv[(vv.STATE=='ALABAMA') & (vv.YEAR>'12-31-2008')]

# Look to see if Indemnity & Severity are correlated

In [ ]:
vv[vv.columns[2:]].describe()

In [ ]:
vv[vv.columns[2:]].corr()

In [ ]:
vv.YEAR.max()